# Pigpen PyTorch Example
by Joe Norton

In [130]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [131]:
import random
from PigpenDataset import PigpenDatasetClass


In [132]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5,), (0.5,))  # Example normalization
])

train_dataset = PigpenDatasetClass('train', transform=transform)
test_dataset = PigpenDatasetClass('test', transform=transform)
valid_dataset = PigpenDatasetClass('validation', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=28, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=28, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=28, shuffle=False)

In [133]:
len(train_dataset), len(test_dataset), len(valid_dataset)

(53974, 13500, 13500)

In [134]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.flatten = nn.Flatten()
        self.conv_relu_stack = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding="same"),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding="same"),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # Adjust these sizes based on your actual output from convolutions
        self.linear_layers = nn.Sequential(
            nn.Linear(1024, 256),  # Adjust size according to output
            nn.ReLU(),
            nn.Linear(256, 28)  # 28 output classes
        )

    def forward(self, x):
        x = self.conv_relu_stack(x)
        x = self.flatten(x)  # Flatten the tensor
        x = self.linear_layers(x)
        return x


Using cpu device


In [135]:

# Create the network, define a loss function, and optimizer
model = SimpleCNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [136]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(valid_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.338125  [   28/53974]
loss: 3.317742  [ 2828/53974]
loss: 3.334797  [ 5628/53974]
loss: 3.334310  [ 8428/53974]
loss: 3.328131  [11228/53974]
loss: 3.331836  [14028/53974]
loss: 3.343597  [16828/53974]
loss: 3.331254  [19628/53974]
loss: 3.328030  [22428/53974]
loss: 3.331958  [25228/53974]
loss: 3.326385  [28028/53974]
loss: 3.335490  [30828/53974]
loss: 3.320841  [33628/53974]
loss: 3.334291  [36428/53974]
loss: 3.327642  [39228/53974]
loss: 3.335960  [42028/53974]
loss: 3.331970  [44828/53974]
loss: 3.340864  [47628/53974]
loss: 3.321266  [50428/53974]
loss: 3.325703  [53228/53974]
Test Error: 
 Accuracy: 3.7%, Avg loss: 3.332238 

Epoch 2
-------------------------------
loss: 3.356669  [   28/53974]
loss: 3.336406  [ 2828/53974]
loss: 3.337413  [ 5628/53974]
loss: 3.332506  [ 8428/53974]
loss: 3.332839  [11228/53974]
loss: 3.319923  [14028/53974]
loss: 3.326802  [16828/53974]
loss: 3.335852  [19628/53974]
loss: 3.340643  [22428/53974]

In [137]:
test(test_loader, model, loss_fn)

Test Error: 
 Accuracy: 3.7%, Avg loss: 3.332134 

